In [29]:
import os

import torch
from torch import nn
from torch import optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [30]:
from nflows.flows.base import Flow
from nflows.distributions.normal import StandardNormal
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation

import tqdm as tqdm
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [31]:
def make_checkpoint(flow, optimizer, loss, filename):
    torch.save({'model_state_dict': flow.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
               }, 
               filename)

def make_flow(d, num_layers = 5):
    base_dist = StandardNormal(shape=[d])
    #base_dist = BoxUniform(low = -2*torch.ones(d), high = 2*torch.ones(d))
    
    transforms = []
    for _ in range(num_layers):
        transforms.append(ReversePermutation(features=d))
        transforms.append(MaskedAffineAutoregressiveTransform(features=d, 
                                                              hidden_features=8))
    transform = CompositeTransform(transforms)
    
    return Flow(transform, base_dist)

def train_flow(flow, optimizer, data, dir, num_iter = 10000):
    losses = np.zeros(num_iter)
    print(-flow.log_prob(inputs=data).mean())
    for i in tqdm.trange(num_iter):
        optimizer.zero_grad()
        loss = -flow.log_prob(inputs=data).mean()
        losses[i] = loss
        
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            np.save(dir + 'losses.npy', losses)
            make_checkpoint(flow, optimizer, loss, dir + 'ckpt_{}'.format(i))
            
    make_checkpoint(flow, optimizer, loss, dir + 'ckpt_{}'.format(num_iter))
    np.save(dir + 'losses.npy', losses)

In [32]:
def marginal_X(X, log_prob):
    X = X.repeat_interleave(101).reshape(-1, 1)
    
    Y = torch.linspace(-0.5, 1.5, 101).reshape(-1, 1)
    Y = Y.repeat(n, 1)
    Y = Y.to(device)

    XY = torch.cat([X, Y], dim = 1)
    LP = flow_XY.log_prob(XY)
    PX = []
    for i in range(n):
        PX += [torch.trapezoid(torch.exp(LP[101 * i : 101 * (i + 1)]), dx = 0.02)]
    return torch.stack(PX, dim = 0)

def marginal_Y(Y, log_prob):
    Y = Y.repeat_interleave(101).reshape(-1, 1)
    
    X = torch.linspace(-0.5, 1.5, 101).reshape(-1, 1)
    X = X.repeat(n, 1)
    X = X.to(device)

    XY = torch.cat([X, Y], dim = 1)
    LP = flow_XY.log_prob(XY)
    PY = []
    for i in range(n):
        PY += [torch.trapezoid(torch.exp(LP[101 * i : 101 * (i + 1)]), dx = 0.02)]
    return torch.stack(PY, dim = 0)

In [33]:
np.random.seed(666)

n = 10000
r = 0

C = stats.multivariate_normal.rvs(np.zeros(2), [[1, r],[r, 1]], size = n)
D = stats.norm.cdf(C)
U = D[:, 0].reshape(n, 1)
V = D[:, 1].reshape(n, 1)

D = torch.tensor(D, dtype = torch.float32).cuda()
U = torch.tensor(U, dtype = torch.float32).cuda()
V = torch.tensor(V, dtype = torch.float32).cuda()
D.to(device)
U.to(device)
V.to(device)

tensor([[0.6844],
        [0.8183],
        [0.4564],
        ...,
        [0.0901],
        [0.0151],
        [0.0290]], device='cuda:0')

In [34]:
flow_XY = make_flow(d = 2)
flow_XY.to(device)
optimizer_XY = optim.Adam(flow_XY.parameters())

os.makedirs('XY', exist_ok = True)
train_flow(flow_XY, optimizer_XY, D, 'XY/')

tensor(5.9043, device='cuda:0', grad_fn=<NegBackward0>)


100%|██████████| 10000/10000 [02:12<00:00, 75.47it/s]


In [6]:
flow_XY.eval();

# Marginals

In [26]:
LPX = torch.log(marginal_X(U, flow_XY.log_prob))

In [27]:
LPY = torch.log(marginal_Y(V, flow_XY.log_prob))

In [28]:
LPX.mean() + LPY.mean() - flow_XY.log_prob(D).mean()

tensor(-0.0019, device='cuda:0', grad_fn=<SubBackward0>)

In [10]:
X = U.repeat_interleave(101).reshape(-1, 1)

In [11]:
Y = torch.linspace(-0.5, 1.5, 101).reshape(-1, 1)
Y = Y.repeat(U.size(0), 1)
Y = Y.to(device)

In [12]:
XY = torch.cat([X, Y], dim = 1)

In [13]:
PP = flow_XY.log_prob(XY)

In [17]:
PX = []
for i in range(n):
    PX += [torch.trapezoid(torch.exp(PP[101 * i : 101 * (i + 1)]), dx = 0.02)]
PX = torch.stack(PX, dim = 0)

In [16]:
torch.stack(PX, dim = 0)

tensor([0.9679, 1.0397, 1.0386,  ..., 0.9619, 0.9605, 0.9676], device='cuda:0',
       grad_fn=<StackBackward0>)

In [ ]:
PX = [torch.trapezoid(torch.exp(PP[101*i:101*(i + 1)]), dx=0.02) for i in range(U.size(0))]

In [ ]:
def prob_X(x):
    x = x*torch.ones(101).reshape(101, 1).to(device)
    y = torch.linspace(-1, 2, 101).reshape(101, 1).to(device)
    xy = torch.cat([x, y], dim = 1).to(device)
    return torch.trapezoid(torch.exp(flow_XY.log_prob(xy)), dx=0.03)

In [ ]:
[print(prob_X(u)) for u in U]

# $p$ Value

In [ ]:
reps = 10000
ss = np.zeros(reps)

for i in range(reps):
    idx = np.random.choice(n, size = n)
    DD = D[idx]
    UU = D[:, 0].reshape(n, 1)
    VV = D[:, 1].reshape(n, 1)

    ss[i] = flow_X.log_prob(UU).mean() + flow_Y.log_prob(VV).mean() - flow_XY.log_prob(DD).mean()
    print(ss[i])

In [ ]:
plt.scatter(S[:, 0], S[:, 1])